<a href="https://colab.research.google.com/github/daonly/personal/blob/main/%5B%EB%8D%B0%EA%B3%BC%EC%9D%B8%5D_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5(%EA%B3%B5%EA%B0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##RNN word embedding 전체 코드
####동영상에서 실습했던 코드입니다. 두 문장의 단어 수가 달라도 학습이 가능하도록 일부 수정했습니다.
####RNN을 이용하면 **'나만의 임베딩 벡터'를 구축할 수 있습니다.** 예를 들어, 연인과 메시지를 주고받을 때 **'토깽이'를 연인의 이름(예: 팜하니) 대신 사용하는 일이 많다면 '토깽이'와 '팜하니'의 코사인 유사도는 1에 가깝게 나올 겁니다.**
####동영상에서 다루지는 않았지만 **일반적으로는 텍스트 데이터를 정제하는 과정을 거칩니다.** 텍스트 데이터의 정제는 일반적으로 형태소 분석기 등을 이용할 수 있습니다. 형태소 분석기 사용 방법은 뒤에 넣어 놓았습니다.

#### 1. 이 셀은 실행만 해 주세요.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.random.set_seed(42)

# one hot vector 시퀀스 만들기 : 건드릴 필요 없음!
def make_ohvs(seq:list, max_len:int):
    ohvs = []
    for idx in seq:
        one_hot_vector = [0 for _ in range(max_len)]
        if idx - 1 > -1:#패딩 아닐 경우에만 1로 지정
            # 지정된 인덱스에 1 설정 (0-based index)
            one_hot_vector[idx - 1] = 1
        ohvs.append(one_hot_vector)

    return ohvs

# 단어장 만들기 : 건들 필요 없음!
def make_wordbook(sentences):
    word_index = dict()
    word_set = set()

    max_len = 0 # 단어 최대 개수
    for s in sentences:
        splitted_s = s.split()
        max_len = max(max_len, len(splitted_s)) # max_len 업데이트
        word_set = word_set.union(set(s.split())) # 단어 집합 업데이트

    i = 1 # i == 0은 패딩이므로 1부터 시작.
    for w in word_set:
        word_index[w] = i
        i += 1

    print("word_index 출력 : ", word_index)

    # input_seqs와 output_seqs 만들기
    input_seqs = []
    output_seqs = []
    for s in sentences:
        cur_s = list()
        for w in s.split():
            cur_s.append(word_index[w])

        input_seqs.append(cur_s)
        output_seqs.append(cur_s[1:])

    # 단어 수 다를 경우 0으로 패딩 작업
    for i in range(len(input_seqs)):
        while len(input_seqs[i]) < max_len:
            input_seqs[i].append(0)
            output_seqs[i].append(0)

    print("input_seqs : ", input_seqs)
    print("output_seqs : ", output_seqs)

    inputs = []
    outputs = []

    for seq in input_seqs:
        ohvs = make_ohvs(seq, len(word_index))

        inputs.append(ohvs)
        outputs.append(ohvs[1:])

    inputs = np.array(inputs)
    outputs = np.array(outputs)
    #print("inputs:", inputs)
    #print("outputs:", outputs)

    return word_index, inputs, outputs

# RNN 본체 : 건들 필요 없음!
def my_RNN(sentences, learning_rate = 0.09, epoch = 2001):
    word_index, inputs, outputs = make_wordbook(sentences)
    wordbook = (word_index, inputs, outputs)
    Ni = len(word_index)       # input dimension
    Nh = Ni       # hidden layer dimension
    No = Ni       # output dimension

    init = tf.initializers.GlorotUniform() # Xavier initializer
    W1 = tf.Variable(init([Ni + Nh, Nh]))
    b1 = tf.Variable(tf.random.normal(shape=(1, Nh)))
    W2 = tf.Variable(init([Nh, No]))
    b2 = tf.Variable(tf.random.normal(shape=(1, No)))

    # 1 epoch마다 위의 learning data들을 모두 학습시켜서 W1, W2, b1, b2를 업데이트함
    cost = 1
    N_training = len(inputs)
    proper_cost = 0.0009 # 적정 cost 수준

    for step in range(epoch):
        for i in range(N_training):# 학습 데이터를 모두 학습시킴
            # i번째 inputs와 outputs를 불러옴
            X = inputs[i]
            Y = outputs[i]

            N_digit = np.size(Y, 0)   # Y numpy array의 행의 개수를 셈.(output 단어 개수만큼 학습해야 하기 때문)
            #print(N_digit)
            h0 = tf.Variable(tf.zeros(shape=(1, Nh))) # hidden layer는 1*Nh의 크기를 가지며 1 epoch마다 초기화함

            for digit in range(N_digit): # input - output 학습
                with tf.GradientTape() as tape:
                    # X_concat X에 hidden layer를 concatnate한 것
                    X_concat = tf.keras.layers.Concatenate()([tf.reshape(X[digit], [1, Ni]), h0])

                    # layer는 X_concat을 W1과 곱한 다음 b1을 더해준 것.
                    layer = tf.sigmoid(tf.matmul(X_concat, W1) + b1)
                    h0 = layer # 이 결과를 다음 값에 hidden layer로 전달해 줌

                    # 현재 자릿수에선 layer 결과에 W2를 곱하고 b2 더한 다음 다시 sigmoid 거쳐서 정답과 맞는지 확인
                    hypothesis = tf.sigmoid(tf.matmul(layer, W2) + b2)

                    # cost function은 cross entropy 형식으로
                    # Y 레이블이 1일 땐 두 번째 term이 죽고, 0일 땐 첫 번째 term이 죽음
                    cost = -tf.reduce_mean(Y[digit]*tf.math.log(hypothesis) + (1-Y[digit])*tf.math.log(1-hypothesis))

                    # 현재 자릿수에서 미분 후 가중치 업데이트
                    dw1, db1, dw2, db2 = tape.gradient(cost, [W1, b1, W2, b2])

                    W1.assign_sub(learning_rate*dw1)
                    b1.assign_sub(learning_rate*db1)
                    W2.assign_sub(learning_rate*dw2)
                    b2.assign_sub(learning_rate*db2)

        if step % 100 == 0:
            print(f"step:{step}, cost:{cost}")
        if cost < proper_cost:
            print(f"step:{step}, cost:{cost}")
            print("적정 수준으로 학습이 완료되었으므로 학습 중단.")
            break

    if cost >= proper_cost:
        print("현재 학습 상태는 cost가 너무 큼! 정답률이 높지 않을 수 있음.")

    matrix = (W1, b1, W2, b2, Ni, Nh, No)
    return wordbook, matrix

#### 2. 여기서 데이터를 수정할 수 있습니다.

In [ ]:
################################################################################
# 1. 데이터 준비

# 여기서 바꿔볼 것은 이 sentences밖에 없습니다. 여러 문장을 넣어 가면서 실험해 보세요.
sentences = ["나는 정말로 호랑이가 좋아요. 호호", "저는 정말로 고양이가 좋아요."]


################################################################################
# 2. RNN 학습
# RNN 학습 : 함수로 만들었습니다.
# learning_rate가 너무 낮으면 학습이 안 되고, 너무 높으면 cost가 제대로 계산이 안 됩니다.
# learning_rate, epoch를 수정해 가면서 실험해 보세요. learning_rate는 학습률, epoch는 학습 횟수를 말합니다.
word_vector = my_RNN(sentences, learning_rate = 0.09, epoch = 2001)[1][0]

word_index 출력 :  {'호호': 1, '호랑이가': 2, '저는': 3, '좋아요.': 4, '고양이가': 5, '정말로': 6, '나는': 7}
input_seqs :  [[7, 6, 2, 4, 1], [3, 6, 5, 4, 0]]
output_seqs :  [[6, 2, 4, 1], [6, 5, 4, 0]]
step:0, cost:0.9917314648628235
step:100, cost:0.13567255437374115
step:200, cost:0.12738555669784546
step:300, cost:0.11858852952718735
step:400, cost:0.10876773297786713
step:500, cost:0.09790223091840744
step:600, cost:0.08993329852819443
step:700, cost:0.08774953335523605
step:800, cost:0.08929500728845596
step:900, cost:0.09197533130645752
step:1000, cost:0.0944875106215477
step:1100, cost:0.09645497798919678
step:1200, cost:0.09788883477449417
step:1300, cost:0.09890465438365936
step:1400, cost:0.09961842000484467
step:1500, cost:0.10011996328830719
step:1600, cost:0.10047324001789093
step:1700, cost:0.10072250664234161
step:1800, cost:0.10089846700429916
step:1900, cost:0.10102133452892303
step:2000, cost:0.10110688954591751
현재 학습 상태는 cost가 너무 큼! 정답률이 높지 않을 수 있음.


##단어 임베딩 벡터 코사인 유사도 확인하기
####'나는', '저는'과 '호랑이가', '고양이가'가 실제로 비슷한지 확인해 봅시다.
####그리고 위 두 문장에서 왜 이 단어 임베딩 벡터들이 비슷하게 나왔는지 생각해 봅시다.(힌트 : RNN은 **순서 기반의 임베딩 벡터 구축 방식**입니다.)
#### 참고 : 코사인 유사도는 두 벡터가 비슷할수록 1에 가까운 값이, 다를수록 -1에 가까운 값이 나옵니다.

In [ ]:
def cosine_similarity(a, b):
    dot_product = tf.tensordot(a, b, axes=1)  # 내적
    norm_a = tf.norm(a)                        # 벡터 A의 크기
    norm_b = tf.norm(b)                        # 벡터 B의 크기
    return dot_product / (norm_a * norm_b)    # 코사인 유사도

# 단어들에 대한 인덱스를 찾아서 입력하세요. 주의 : 위의 word_index 값에서 -1씩 해야 합니다.
# 즉, '호랑이가' : 1, '고양이가' : 5라면 word_vector[0], word_vector[4]라고 적어야 합니다.
similarity = cosine_similarity(word_vector[1], word_vector[4])
print(f"Cosine Similarity: {similarity.numpy()}")

Cosine Similarity: 0.9609073996543884


## RNN으로 문서 분류하기
#### RNN은 '오늘 날씨 어때?'라는 문장이 있다면 하나씩 집어 넣으면서 예측하도록 하는 것이라고 했습니다. 문서 분류도 이를 그대로 활용할 수 있습니다.
#### 즉, **문장 '오늘 날씨 어때?' 다음에 '날씨'라는 단어(정답)를 하나 추가하는 겁니다.** (하나의 문장도 하나의 문서라고 볼 수 있습니다.) **그렇다면 우리의 모델은 '오늘 날씨 어때?'라는 문장이 왔을 때 '날씨'라는 '단어(정답)'를 예측할 겁니다.**
#### (사실 실제로는 RNN 모델을 조금 수정해서 하는 것이 더 바람직하지만, 쉬운 이해와 간단한 실습을 위해 이렇게 진행합니다.)
#### 단, 이런 방식으로 하려면 '정답'에 해당하는 단어가 학습 문장과 겹치면 안 되겠지요?
#### 위에서 만든 my_RNN으로 문서 분류를 해 봅시다.(참고로, 감성 분석도 문서 분류의 하나로 간주할 수 있습니다.(정도성을 갖지 않는다면))

####1. 이 셀은 실행만 해 주세요.

In [ ]:
def concate_labels(sentences, labels):# 학습할 문장과 정답을 하나의 문장으로 합치는 함수
    concated = []

    if len(sentences) != len(labels):
        print("문장과 정답의 개수가 일치하지 않습니다.")
    else:
        for i in range(len(sentences)):
            concated.append(sentences[i] + " " + labels[i])

    return concated

# 여기선 W1, W2, b1, b2를 업데이트하지 않고, 위에서 학습한 것을 그대로 사용함
def my_RNN_predict(test_sentences, learned):
    correct = 0
    incorrect = 0
    N_test = len(test_sentences)

    wordbook = learned[0]
    matrix = learned[1]

    # test_sentences의 문장들을 wordbook의 인덱스 데이터로 바꾸기
    input_seqs = []
    for ts in test_sentences:
        splitted_ts = ts.split()
        print(splitted_ts)
        seq = []
        for w in splitted_ts:
            seq.append(wordbook[0][w])
        input_seqs.append(seq)

    # 단어 수 다를 경우 0으로 패딩 작업
    max_len =  max(len(x) for x in input_seqs)

    for i in range(len(input_seqs)):
        while len(input_seqs[i]) < max_len:
            input_seqs[i].append(0)

    print("test input_seqs : ", input_seqs)

    #test input 만들기
    inputs = []
    for seq in input_seqs:
        ohvs = make_ohvs(seq, len(wordbook[0]))
        inputs.append(ohvs)

    inputs = np.array(inputs)
    #print("test inputs : ", inputs)
    #print("정답 인덱스 : ", wordbook[0]['기쁨'], wordbook[0]['분노'])

    pred_result_v = []
    for i in range(N_test):# 테스트
            # i번째 input를 불러옴
            X = inputs[i]

            try:
                N_digit = input_seqs[i].index(0)
            except ValueError:
                N_digit = np.size(X, 0)
            #N_digit = np.size(X, 0)
            #print("limit idx : ", N_digit)
            W1 = matrix[0]
            b1 = matrix[1]
            W2 = matrix[2]
            b2 = matrix[3]
            Ni = matrix[4]
            Nh = matrix[5]
            No = matrix[6]
            h0 = tf.Variable(tf.zeros(shape=(1, Nh))) # hidden layer는 1*Nh의 크기를 가지며 1 epoch마다 초기화함

            predicted = []
            for digit in range(N_digit): # input - output 예측
                with tf.GradientTape() as tape:
                    # X_concat X에 hidden layer를 concatnate한 것
                    X_concat = tf.keras.layers.Concatenate()([tf.reshape(X[digit], [1, Ni]), h0])

                    # layer는 X_concat을 W1과 곱한 다음 b1을 더해준 것.
                    layer = tf.sigmoid(tf.matmul(X_concat, W1) + b1)
                    h0 = layer # 이 결과를 다음 값에 hidden layer로 전달해 줌

                    # 현재 자릿수에선 layer 결과에 W2를 곱하고 b2 더한 다음 다시 sigmoid 거쳐서 정답과 맞는지 확인
                    hypothesis = tf.sigmoid(tf.matmul(layer, W2) + b2)

                    # test에선 가중치를 업데이트하지 않음
                    predicted.append(tf.cast(hypothesis > 0.5, dtype=tf.float32).numpy())

            # 마지막 것만 가져옴(마지막이 label이기 때문)
            pred_v = predicted[-1].tolist()[0]
            pred_v = [int(x) for x in pred_v]

            pred_result_v.append(pred_v)

    pred_result = []
    for pr in pred_result_v:
        if pr.count(1) != 1:
            pred_result.append("오류")
        elif pr[wordbook[0]['기쁨'] - 1] == 1:
            pred_result.append("기쁨")
        elif pr[wordbook[0]['분노'] - 1] == 1:
            pred_result.append("분노")

    return pred_result

####2. 먼저 train 데이터 셋으로 학습시킵니다. 조금 시간이 걸릴 수 있습니다.
####너무 오래 걸린다고 생각하면 epoch(작게)와 learning_rate(크게)를 조정해 보세요.

In [ ]:
################################################################################
# 1. 학습 데이터 준비
# 학습 데이터
train_sentences = ["나는 호랑이가 참 좋아요.",
                   "정말로 호랑이가 포효해요.",
                   "호랑이가 포효해요.",
                   "나는 정말로 호랑이가 좋아요.",
                   "나는 호랑이가 좋아요.",

                   "나는 독수리가 참 좋아요.",
                   "정말로 독수리가 하늘을 날아요.",
                   "독수리가 하늘을 날아요.",
                   "나는 정말로 독수리가 좋아요.",
                   "나는 독수리가 좋아요.",
                   ]
 # 정답
labels = ["기쁨", "기쁨", "기쁨","기쁨", "기쁨",
          "분노", "분노", "분노", "분노", "분노",]

################################################################################
# 2. 학습 데이터 변환 : 학습 데이터 끝에 정답 붙이기
concated_sentences = concate_labels(train_sentences, labels)
print(concated_sentences)

################################################################################
# 3. RNN 학습
learned = my_RNN(concated_sentences, learning_rate = 0.09)

['나는 호랑이가 참 좋아요. 기쁨', '정말로 호랑이가 포효해요. 기쁨', '호랑이가 포효해요. 기쁨', '나는 정말로 호랑이가 좋아요. 기쁨', '나는 호랑이가 좋아요. 기쁨', '나는 독수리가 참 좋아요. 분노', '정말로 독수리가 하늘을 날아요. 분노', '독수리가 하늘을 날아요. 분노', '나는 정말로 독수리가 좋아요. 분노', '나는 독수리가 좋아요. 분노']
word_index 출력 :  {'기쁨': 1, '호랑이가': 2, '날아요.': 3, '독수리가': 4, '포효해요.': 5, '참': 6, '하늘을': 7, '분노': 8, '좋아요.': 9, '정말로': 10, '나는': 11}
input_seqs :  [[11, 2, 6, 9, 1], [10, 2, 5, 1, 0], [2, 5, 1, 0, 0], [11, 10, 2, 9, 1], [11, 2, 9, 1, 0], [11, 4, 6, 9, 8], [10, 4, 7, 3, 8], [4, 7, 3, 8, 0], [11, 10, 4, 9, 8], [11, 4, 9, 8, 0]]
output_seqs :  [[2, 6, 9, 1], [2, 5, 1, 0], [5, 1, 0, 0], [10, 2, 9, 1], [2, 9, 1, 0], [4, 6, 9, 8], [4, 7, 3, 8], [7, 3, 8, 0], [10, 4, 9, 8], [4, 9, 8, 0]]
step:0, cost:0.7780826091766357
step:100, cost:0.07597722113132477
step:200, cost:0.06968691200017929
step:300, cost:0.05009394884109497
step:400, cost:0.028331100940704346
step:500, cost:0.016982728615403175
step:600, cost:0.011266296729445457
step:700, cost:0.007909804582595825
step:800, cost:0.005787574

####3. 다음으로 모델이 보지 못했던 새로운 문장으로 예측을 실시해 봅니다.
#### 확인을 위해 모델이 봤던 문장도 넣었습니다.
#### 학습 데이터가 워낙 적기 때문에 오류가 발생하거나 정답률이 낮을 수 있습니다.

In [ ]:

################################################################################
# 4. 테스트 데이터 준비 및 테스트
test_sentences = ["나는 정말로 호랑이가 좋아요.", # 원래 있는 문장
                  "나는 정말로 독수리가 좋아요.", # 원래 있는 문장
                  "호랑이가 좋아요.",
                  "독수리가 좋아요.",
                  "정말로 나는 호랑이가 좋아요.",
                  "정말로 나는 독수리가 좋아요.",
                  ]

predict = my_RNN_predict(test_sentences, learned)

print(predict)

['나는', '정말로', '호랑이가', '좋아요.']
['나는', '정말로', '독수리가', '좋아요.']
['호랑이가', '좋아요.']
['독수리가', '좋아요.']
['정말로', '나는', '호랑이가', '좋아요.']
['정말로', '나는', '독수리가', '좋아요.']
test input_seqs :  [[11, 10, 2, 9], [11, 10, 4, 9], [2, 9, 0, 0], [4, 9, 0, 0], [10, 11, 2, 9], [10, 11, 4, 9]]
['기쁨', '분노', '기쁨', '분노', '기쁨', '분노']


##다양한 패키지 소개 및 활용 방법

###1. 형태소 분석기
####형태소 분석기를 활용하면 특정 어미만 추출하거나 텍스트를 정제하는 데 사용할 수 있습니다.
####예를 들어, '듣다'는 **'듣다, 들어, 들으니, 들었는데' 등 다양한 형태**로 활용되는데, **형태소 분석기를 이용하면 이들이 모두 한 단어로 인식되도록 정제할 수 있습니다**.
####여기서는 정확도가 높다고 알려진 **Kiwipiepy 형태소 분석기**를 실습해 봅니다.

In [ ]:
!pip install kiwipiepy

#### 먼저 한 문장에서 형태소 분석을 실시해 봅시다.
#### **N 계열은 체언, V 계열은 용언, M 계열은 관형사, 부사, J 계열은 조사, E 계열은 어미, X 계열은 접사**입니다.
#### 이를 활용한다면 문서에서 품사별로 그 개수도 확인할 수 있겠죠?
#### 각각의 tag에 대한 자세한 분류는 다음 페이지를 참조하세요: https://github.com/bab2min/kiwi

In [ ]:
from kiwipiepy import Kiwi
sentence = "나는 고양이가 좋아용.^^ㅋㅋㅋ"
print(sentence)                         # 원래 문장을 출력합니다.
kiwi = Kiwi()                           # 형태소 분석기를 'kiwi'라는 이름으로 로드합니다.
morph_result = kiwi.tokenize(sentence)  # 형태소 분석을 실시합니다.
print(type(morph_result))               # 결과는 리스트로 반환됩니다.
print(type(morph_result[0]))            # 하나의 토큰은 'kiwipiepy.Token'이라는 객체로 반환됩니다.
print(morph_result[0].form)             # 토큰 안의 데이터는 '.'으로 접근합니다.


# 이 부분이 [Token(form='나', ....)] 부분입니다.
morph_result

나는 고양이가 좋아용.^^ㅋㅋㅋ
<class 'list'>
<class 'kiwipiepy.Token'>
나


[Token(form='나', tag='NP', start=0, len=1),
 Token(form='는', tag='JX', start=1, len=1),
 Token(form='고양이', tag='NNG', start=3, len=3),
 Token(form='가', tag='JKS', start=6, len=1),
 Token(form='좋', tag='VA', start=8, len=1),
 Token(form='어요', tag='EF', start=9, len=2),
 Token(form='ᆼ', tag='Z_CODA', start=10, len=1),
 Token(form='.', tag='SF', start=11, len=1),
 Token(form='^^', tag='SW', start=12, len=2),
 Token(form='ㅋㅋㅋ', tag='SW', start=14, len=3)]

#### 여러 문장에서 '좋다'가 있는 단어와, 그 전체 문장만 추출해 봅시다.
#### 특정 단어를 찾을 땐 **형태소 분석 결과를 기준으로 하는 것이 좋습니다.** 겉으로 드러난 형태는 비슷하지만 형태소는 다를 수 있기 떄문입니다.(예: '나+는'의 축약형 '난'과 '나다'의 활용형 '난')
#### 여기서는 동사를 의미하는 'VA'를 이용해 찾아보겠습니다.

In [ ]:
data = ['나는 고려대가 좋다',
        '난 어렸을 땐 동생이 좋았는데, 지금은 참 웬수야.',
        '우리 가족은 오늘 육거리 시장에 갔다.',
        '좋은 사람 만나 웃으며 더 사랑받으며 살아',
        '이야, 군대에서 기술을 배우다니. 좋았겠네.',
        '아기 상어, 뚜 루루 뚜루, 귀여운, 뚜 루루 뚜루, 바닷속, 뚜 루루 뚜루, 아기 상어!'
        ]

data_size = len(data)                           # data의 각 원소들이 지금은 한 문장씩이지만, 각각이 하나의 '문서'라고 생각할 수도 있습니다.
i = 0
while i < data_size:
    morph_result = kiwi.tokenize(data[i])
    for t in morph_result:
        if t.tag == 'VA' and t.form == '좋':    # 'VA'인 태그이고, 형태소가 '좋'이라면
            # 1. 그 단어를 출력
            start = t.start                     # '좋다' 단어 시작 지점
            end = data[i][start:].find(' ')     # 띄어쓰기로 '좋다' span 찾기 : find는 찾으면 해당 인덱스를, 못 찾으면 -1을 반환
            print(data[i][start:end+start], end = '') if end != -1 else print(data[i][start:], end = '')

            # 2. 그 문장을 출력
            print(' : ' + data[i])
    i += 1

좋다 : 나는 고려대가 좋다
좋았는데, : 난 어렸을 땐 동생이 좋았는데, 지금은 참 웬수야.
좋은 : 좋은 사람 만나 웃으며 더 사랑받으며 살아
좋았겠네. : 이야, 군대에서 기술을 배우다니. 좋았겠네.


####이번엔 단어별 빈도를 세어 많이 사용된 순으로 나열해 봅시다. 방식은 바로 위에서 '좋다'를 센 것과 비슷합니다.
####여기서는 form과 tag를 기준으로 단어의 빈도를 세어 보겠습니다.

In [ ]:
data = ['나는 고려대가 좋다',
        '난 어렸을 땐 동생이 좋았는데, 지금은 참 웬수야.',
        '우리 가족은 오늘 육거리 시장에 갔다.',
        '좋은 사람 만나 웃으며 더 사랑받으며 살아',
        '이야, 군대에서 기술을 배우다니. 좋았겠네.',
        '아기 상어, 뚜 루루 뚜루, 귀여운, 뚜 루루 뚜루, 바닷속, 뚜 루루 뚜루, 아기 상어!'
        ]

data_size = len(data)                           # data의 각 원소들이 지금은 한 문장씩이지만, 각각이 하나의 '문서'라고 생각할 수도 있습니다.
i = 0

words_freq = dict()

while i < data_size:
    morph_result = kiwi.tokenize(data[i])
    for t in morph_result:
        word = (t.form, t.tag)
        if word not in words_freq:
            words_freq[word] = 1
        else:
            words_freq[word] += 1
    i += 1

sorted_wf = dict(sorted(words_freq.items(), key=lambda item: item[1], reverse=True)) # 딕셔너리 value 기준 내림차순

for k, v in sorted_wf.items():
        print(k, v)

(',', 'SP') 8
('좋', 'VA') 4
('었', 'EP') 4
('.', 'SF') 4
('뚜', 'MAG') 3
('루루', 'NNG') 3
('뚜루', 'NNG') 3
('나', 'NP') 2
('다', 'EF') 2
('ᆫ', 'JX') 2
('은', 'JX') 2
('은', 'ETM') 2
('으며', 'EC') 2
('아기', 'NNG') 2
('상어', 'NNG') 2
('는', 'JX') 1
('고려대', 'NNP') 1
('가', 'JKS') 1
('어리', 'VA') 1
('을', 'ETM') 1
('때', 'NNG') 1
('동생', 'NNG') 1
('이', 'JKS') 1
('는데', 'EC') 1
('지금', 'NNG') 1
('참', 'MAG') 1
('웬수', 'NNG') 1
('이', 'VCP') 1
('야', 'EF') 1
('우리', 'NP') 1
('가족', 'NNG') 1
('오늘', 'MAG') 1
('육', 'NR') 1
('거리', 'NNG') 1
('시장', 'NNG') 1
('에', 'JKB') 1
('가', 'VV') 1
('사람', 'NNG') 1
('만나', 'VV') 1
('어', 'EC') 1
('웃', 'VV-R') 1
('더', 'MAG') 1
('사랑', 'NNG') 1
('받', 'VV-R') 1
('살', 'VV') 1
('어', 'EF') 1
('이야', 'IC') 1
('군대', 'NNG') 1
('에서', 'JKB') 1
('기술', 'NNG') 1
('을', 'JKO') 1
('배우', 'VV') 1
('다니', 'EF') 1
('겠', 'EP') 1
('네', 'EF') 1
('귀엽', 'VA-I') 1
('바닷속', 'NNG') 1
('!', 'SF') 1


####결과를 보면 '.'은 8번, '었'은 4번이 나오는 등 기호나 문법적 요소가 상위에 있죠? 이런 이유로 이러한 것들은 포함되지 않도록 일반적으로는 불용어 리스트를 작성해서 결과에 포함되지 않도록 합니다.
####하지만 여기서는 불용어 리스트를 작성하는 대신 **명사와 동사류만 뽑아서 출력해 보겠습니다.**

In [ ]:
for k, v in sorted_wf.items():
    if k[1][0] == 'N' or k[1][0] == 'V':    #tag가 'N' 또는 'V'로 시작하는 경우에만 출력
        print(k, v)

('좋', 'VA') 4
('루루', 'NNG') 3
('뚜루', 'NNG') 3
('나', 'NP') 2
('아기', 'NNG') 2
('상어', 'NNG') 2
('고려대', 'NNP') 1
('어리', 'VA') 1
('때', 'NNG') 1
('동생', 'NNG') 1
('지금', 'NNG') 1
('웬수', 'NNG') 1
('이', 'VCP') 1
('우리', 'NP') 1
('가족', 'NNG') 1
('육', 'NR') 1
('거리', 'NNG') 1
('시장', 'NNG') 1
('가', 'VV') 1
('사람', 'NNG') 1
('만나', 'VV') 1
('웃', 'VV-R') 1
('사랑', 'NNG') 1
('받', 'VV-R') 1
('살', 'VV') 1
('군대', 'NNG') 1
('기술', 'NNG') 1
('배우', 'VV') 1
('귀엽', 'VA-I') 1
('바닷속', 'NNG') 1


###2. Gensim
#### Word2Vec, FastText 등을 직접 학습시키고 활용할 수 있습니다. 여기서는 Word2Vec을 사용해 봅니다.
#### 여기서는 **Kiwi 형태소 분석기를 이용해 텍스트를 토큰화한 뒤에 학습**시킵니다.
####토큰화하면 텍스트를 그대로 사용하는 것보다 **좋은 품질의 임베딩 벡터를 얻을 수 있습니다.**(단, 토큰화가 '텍스트 전처리'의 모든 과정을 거치는 것은 아니라는 점 주의해주세요.)
#### 사실 Word2Vec이나 FastText 같은 분포 기반의 구축 방법보다는 RNN 등의 순서 기반의 구축 방법이 일반적으로는 더 우수한 품질의 임베딩 벡터를 만들어 냅니다.

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

# 학습시킬 데이터
data = ['나는 고려대가 좋다',
        '난 어렸을 땐 동생이 좋았는데, 지금은 참 웬수야.',
        '우리 가족은 오늘 육거리 시장에 갔다.',
        '좋은 사람 만나 웃으며 더 사랑받으며 살아',
        '네가 포기하지 않는다면 좋을 거야.',
        '아기 상어, 뚜 루루 뚜루, 귀여운, 뚜 루루 뚜루, 바닷속, 뚜 루루 뚜루, 아기 상어!'
        ]

# 데이터 형태소 분석(토큰화) : 원래는 한글 외 다른 문자 제거 및 불용어 제거 등 전처리 과정을 겪지만 여기서는 바로 토큰화를 진행하겠습니다.
sentences = [[t.form for t in kiwi.tokenize(s)] for s in data]
print('토큰화된 결과 : ')
for s in sentences:
    print(s)


# sentences: 학습할 문장(토큰화된 단어 리스트).
# vector_size: 단어 벡터의 차원 수.
# window: 중심 단어를 기준으로 양옆의 몇 개 단어를 고려할지. (여기서는 3개의 주변 단어를 고려)
# min_count: 단어의 최소 등장 횟수. 이 값보다 적게 등장하는 단어는 무시됨.
# workers: CPU의 병렬 처리 수.
# sg: 1이면 Skip-Gram을 사용하고, 0이면 CBOW를 사용함.

# Word2Vec 모델 생성 및 훈련(실제로는 512차원 등 보다 높은 차원으로 진행합니다)
model = Word2Vec(sentences, vector_size=10, window=15, min_count=2, workers=12, sg=1)

# 단어 벡터 확인
# 주의 : '고려대' 같은 경우는 한 번밖에 나오지 않았기 때문에 학습되지 않습니다. 한 번 나온 단어도 학습시키려면 min_count를 1로 하면 됩니다.
word = '상어'
vector = model.wv[word]
print('\n\n')
print(f'{word} 벡터 확인 :', vector)

토큰화된 결과 : 
['나', '는', '고려대', '가', '좋', '다']
['나', 'ᆫ', '어리', '었', '을', '때', 'ᆫ', '동생', '이', '좋', '었', '는데', ',', '지금', '은', '참', '웬수', '이', '야', '.']
['우리', '가족', '은', '오늘', '육', '거리', '시장', '에', '가', '었', '다', '.']
['좋', '은', '사람', '만나', '어', '웃', '으며', '더', '사랑', '받', '으며', '살', '어']
['너', '가', '포기', '하', '지', '않', '는다면', '좋', '을', '거', '이', '야', '.']
['아기', '상어', ',', '뚜', '루루', '뚜루', ',', '귀엽', '은', ',', '뚜', '루루', '뚜루', ',', '바닷속', ',', '뚜', '루루', '뚜루', ',', '아기', '상어', '!']



상어 벡터 확인 : [-0.03711143 -0.08822143  0.05469841  0.06491257 -0.00787101 -0.0667427
 -0.07050335 -0.02579085  0.05167998 -0.03664207]
